# <center>
<div style="
    background: #f0f333ff;
    border-left: 5px solid #ecd242ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h1 style="
        text-align: center;
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 600;
    ">
    CA Scheduling
    </h1>
</div>

##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    First Test : with chat GPT
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Basic example
    </h3>
</div>

In [39]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Paramètres du problème
    # -------------------------
    employees = ["A", "B", "C", "D"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    shifts = ["Matin", "Après-midi"]
    activities = ["Telephone", "Derogation", "Reclamation"]

    n_emp = len(employees)
    n_days = len(days)
    n_shifts = len(shifts)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for s in range(n_shifts):
                for a in range(len(activities)):
                    x[e, d, s, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{shifts[s]}_{activities[a]}")

    # -------------------------
    # Contraintes
    # -------------------------

    # 1️⃣ Chaque créneau : 2 Téléphones, 1 Dérogation, 1 Réclamation
    for d in range(n_days):
        for s in range(n_shifts):
    #         # Chaque activité a le bon nombre de personnes
            model.Add(sum(x[e, d, s, 0] for e in range(n_emp)) == 2)  # Téléphone
            model.Add(sum(x[e, d, s, 1] for e in range(n_emp)) == 1)  # Dérogation
            model.Add(sum(x[e, d, s, 2] for e in range(n_emp)) == 1)  # Réclamation

            # Chaque employé fait exactement 1 activité par créneau
            for e in range(n_emp):
                model.Add(sum(x[e, d, s, a] for a in range(len(activities))) == 1)

    # 2️⃣ Pas plus de 3 jours de Téléphone par semaine par employé
    # Un jour est compté Téléphone s'il fait téléphone matin ou après-midi
    for e in range(n_emp):
        is_tel_day = []
        for d in range(n_days):
            tel_day = model.NewBoolVar(f"tel_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) >= 1).OnlyEnforceIf(tel_day)
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) == 0).OnlyEnforceIf(tel_day.Not())
            is_tel_day.append(tel_day)
        model.Add(sum(is_tel_day) <= 3)

    # 3️⃣ Pas 3 jours consécutifs de Téléphone
    for e in range(n_emp):
        for d in range(n_days - 2):
            model.Add(sum(x[e, d + i, s, 0] for i in range(3) for s in range(n_shifts)) <= 4)
            # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    solver.parameters.num_search_workers = 8

    result = solver.Solve(model)

    # -------------------------
    # Affichage du planning
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for d in range(n_days):
            print(f"\n=== {days[d]} ===")
            for s in range(n_shifts):
                print(f"  {shifts[s]} :")
                for e in range(n_emp):
                    for a in range(len(activities)):
                        if solver.Value(x[e, d, s, a]) == 1:
                            print(f"    {employees[e]} → {activities[a]}")
    else:
        print("❌ Aucune solution trouvée.")

if __name__ == "__main__":
    main()


=== Lundi ===
  Matin :
    A → Derogation
    B → Telephone
    C → Reclamation
    D → Telephone
  Après-midi :
    A → Derogation
    B → Telephone
    C → Reclamation
    D → Telephone

=== Mardi ===
  Matin :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone
  Après-midi :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone

=== Mercredi ===
  Matin :
    A → Telephone
    B → Telephone
    C → Derogation
    D → Reclamation
  Après-midi :
    A → Telephone
    B → Telephone
    C → Reclamation
    D → Derogation

=== Jeudi ===
  Matin :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone
  Après-midi :
    A → Telephone
    B → Derogation
    C → Telephone
    D → Reclamation

=== Vendredi ===
  Matin :
    A → Telephone
    B → Telephone
    C → Reclamation
    D → Derogation
  Après-midi :
    A → Derogation
    B → Telephone
    C → Telephone
    D → Reclamation


###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Add more constraints
    </h3>
</div>

In [26]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Paramètres du problème
    # -------------------------
    employees = ["A", "B", "C", "D"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    shifts = ["Matin", "Après-midi"]
    activities = ["Tél", "Rens", "Dérog", "Récla"]

    n_emp = len(employees)
    n_days = len(days)
    n_shifts = len(shifts)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for s in range(n_shifts):
                for a in range(len(activities)):
                    x[e, d, s, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{shifts[s]}_{activities[a]}")

    # -------------------------
    # Contraintes
    # -------------------------

    # 1️⃣ Chaque créneau : 2 Téléphones, 1 Dérogation, 1 Réclamation
    for d in range(n_days):
        for s in range(n_shifts):
            # Chaque activité a le bon nombre de personnes
            model.Add(sum(x[e, d, s, 0] for e in range(n_emp)) == 2)  # Téléphone
            model.Add(sum(x[e, d, s, 1] for e in range(n_emp)) == 1)  # Dérogation
            model.Add(sum(x[e, d, s, 2] for e in range(n_emp)) == 1)  # Réclamation

            # Chaque employé fait exactement 1 activité par créneau
            for e in range(n_emp):
                model.Add(sum(x[e, d, s, a] for a in range(len(activities))) == 1)

    # 2️⃣ Pas plus de 3 jours de Téléphone par semaine par employé
    # Un jour est compté Téléphone s'il fait téléphone matin ou après-midi
    for e in range(n_emp):
        is_tel_day = []
        for d in range(n_days):
            tel_day = model.NewBoolVar(f"tel_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) >= 1).OnlyEnforceIf(tel_day)
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) == 0).OnlyEnforceIf(tel_day.Not())
            is_tel_day.append(tel_day)
        model.Add(sum(is_tel_day) <= 3)

    # 3️⃣ Pas 3 jours consécutifs de Téléphone
    for e in range(n_emp):
        for d in range(n_days - 2):
            model.Add(sum(x[e, d + i, s, 0] for i in range(3) for s in range(n_shifts)) <= 4)
            # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

    # 4️⃣ Pas 2 jours consécutifs de Dérogation
    for e in range(n_emp):
        for d in range(n_days - 1):
            # Jour d : il fait dérogation ?
            derog_d = model.NewBoolVar(f"derog_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 1] for s in range(n_shifts)) >= 1).OnlyEnforceIf(derog_d)
            model.Add(sum(x[e, d, s, 1] for s in range(n_shifts)) == 0).OnlyEnforceIf(derog_d.Not())

            # Jour suivant : dérogation ?
            derog_next = model.NewBoolVar(f"derog_day_{employees[e]}_{days[d+1]}")
            model.Add(sum(x[e, d+1, s, 1] for s in range(n_shifts)) >= 1).OnlyEnforceIf(derog_next)
            model.Add(sum(x[e, d+1, s, 1] for s in range(n_shifts)) == 0).OnlyEnforceIf(derog_next.Not())

            # Pas deux jours consécutifs de dérogation
            model.AddBoolOr([derog_d.Not(), derog_next.Not()])

    # 5️⃣ Pas 2 jours consécutifs de Réclamation
    for e in range(n_emp):
        for d in range(n_days - 1):
            # Jour d : fait-il réclamation ?
            recl_d = model.NewBoolVar(f"reclam_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 2] for s in range(n_shifts)) >= 1).OnlyEnforceIf(recl_d)
            model.Add(sum(x[e, d, s, 2] for s in range(n_shifts)) == 0).OnlyEnforceIf(recl_d.Not())

            # Jour suivant : fait-il réclamation ?
            recl_next = model.NewBoolVar(f"reclam_day_{employees[e]}_{days[d+1]}")
            model.Add(sum(x[e, d+1, s, 2] for s in range(n_shifts)) >= 1).OnlyEnforceIf(recl_next)
            model.Add(sum(x[e, d+1, s, 2] for s in range(n_shifts)) == 0).OnlyEnforceIf(recl_next.Not())

            # Pas deux jours consécutifs de réclamation
            model.AddBoolOr([recl_d.Not(), recl_next.Not()])

    # 6️⃣ Objectif : minimiser les changements d'activité entre matin et après-midi
    penalties = []
    for e in range(n_emp):
        for d in range(n_days):
            # Variable binaire : 1 si les activités matin et après-midi sont différentes
            different_activity = model.NewBoolVar(f"different_{employees[e]}_{days[d]}")

            # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
            same_activity_bools = []
            for a in range(len(activities)):
                both_same = model.NewBoolVar(f"same_{employees[e]}_{days[d]}_{activities[a]}")
                model.AddBoolAnd([x[e, d, 0, a], x[e, d, 1, a]]).OnlyEnforceIf(both_same)
                model.AddBoolOr([x[e, d, 0, a].Not(), x[e, d, 1, a].Not()]).OnlyEnforceIf(both_same.Not())
                same_activity_bools.append(both_same)

            # Si au moins une activité est identique sur la journée, alors different_activity = 0
            model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
            # Sinon, different_activity = 1
            model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

            penalties.append(different_activity)

    # On minimise le nombre total de changements d'activité sur la semaine
    model.Minimize(sum(penalties))

    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    solver.parameters.num_search_workers = 8

    result = solver.Solve(model)

    # -------------------------
    # Affichage du planning
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for e in range(n_emp):
            st = f"{employees[e]}  →  "
            for d in range(n_days):
                for s in range(n_shifts):
                    for a in range(len(activities)):
                        if solver.Value(x[e, d, s, a]) == 1:
                            st += f"{activities[a]:6} "
            print(st)
    else:
        print("❌ Aucune solution trouvée.")

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")

if __name__ == "__main__":
    main()


A  →  Rens   Rens   Tél    Tél    Tél    Tél    Rens   Rens   Tél    Tél    
B  →  Dérog  Dérog  Tél    Tél    Rens   Rens   Tél    Tél    Rens   Rens   
C  →  Tél    Tél    Rens   Rens   Dérog  Dérog  Tél    Tél    Dérog  Dérog  
D  →  Tél    Tél    Dérog  Dérog  Tél    Tél    Dérog  Dérog  Tél    Tél    

Statistics
  - conflicts      : 0
  - branches       : 446
  - wall time      : 0.051749 s


##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Second Test : CA Planning with one shift a day
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    test
    </h3>
</div>

#### **Constraints**

1. An employee must have exactly one activity per day

2. Number of employees per activity
- Number of Tél >= 5
- Number of Rens >= 3
- Number of Dérog >= 1
- Number of Récla == 1
- Number of Imp == 1
- Number of Libre <= 1

3. Limitation on consecutive days on one activity
- Not 3 consecutive days on "Tél"
- Not 4 days on "Tél" during the week
- Not 2 consecutive days on "Rens"
- Not 3 days on "Rens" during the week
- No more than 1 day on "Récla", "Dérog" or "Imp"

4. 



In [16]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, employees, days, activities, tasks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._employees = employees
        self._days = days
        self._activities = activities
        self._tasks = tasks
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        print("       " + "     ".join(self._employees))
        for d in self._days:
            st = f"{d + 1:2} → "
            for e in self._employees:
                is_working = False
                for a in self._activities:
                    if self.value(self._tasks[(e, d, a)]):
                        st += f"{a:5} "
                        is_working = True
                if not is_working:
                    st += "  *   "
            print(st)
            if (d + 1) % 5 == 0:
                print()
        print()
        self.print_employees_statistics()
        print()
        self.print_days_statistics()
        print()
    
        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def print_days_statistics(self):
        for d in self._days:
            st = f"{d + 1:2} → "
            total = 0
            for a in self._activities:
                nb = sum(self.value(self._tasks[(e, d, a)]) for e in self._employees)
                total += nb
                st += f"{a}:{nb} "
            print(st + f" (nb activities: {total})")

    def print_employees_statistics(self):
        for e in self._employees:
            st = f"{e} → "
            total = 0
            for a in self._activities:
                nb = sum(self.value(self._tasks[(e, d, a)]) for d in self._days)
                total += nb
                st += f"{a}:{nb} "
            print(st + f" (nb activities: {total})")

    def solutionCount(self):
        return self._solution_count


# Data.
n_employees = 14
n_weeks = 2
n_days = 5 * n_weeks

employees = [chr(i) for i in range(65, 65 + n_employees)]
weeks = range(n_weeks)
days = range(n_days)
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp", "Libre"]

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# tasks[(e, d, s, a)]: employee 'e' works day 'd' on shift 's' on activity 'a'.
tasks = {}
for e in employees:
    for d in days:
        for a in activities:
            tasks[(e, d, a)] = model.new_bool_var(f"task_{e}_{d}_{a}")



# ------------------- #
#                     #
#     CONSTRAINTS     #
#                     #
# ------------------- #

# Days Off
is_present = {item : [True] * len(days) for item in employees}
is_present['A'][0] = False
is_present['A'][1] = False
is_present['B'][9] = False
is_present['C'][3] = False
is_present['F'][0] = False
is_present['H'][2] = False
# is_present['O'] = [False] * len(days) * len(shifts)

# 1️⃣ Each employee must have exactly one activity per shift.
for e in employees:
    for d in days:
        if is_present[e][d]:
            model.add(sum(tasks[(e, d, a)] for a in activities) == 1)
        else:
            # Employee is not present this day
            model.add(sum(tasks[(e, d, a)] for a in activities) == 0)

# 2️⃣ Minimum / Maximum number of employees per activity.
# To be defined according to how many people are available.
for d in days:
    # Minimum number of Tél >= 5
    model.add(sum(tasks[(e, d, activities[0])] for e in employees) >= 5)
    # Minimum number of Rens >= 3
    model.add(sum(tasks[(e, d, activities[1])] for e in employees) >= 3)
    # Minimum number of Dérog >= 1
    model.add_at_least_one(tasks[(e, d, activities[2])] for e in employees)
    # Minimum number of Récla == 1
    model.add_exactly_one(tasks[(e, d, activities[3])] for e in employees)
    # model.Add(sum(tasks[(e, d, activities[3])] for e in employees) == 1)
    # Minimum number of Imp == 1
    model.add_exactly_one(tasks[(e, d, activities[4])] for e in employees)
    # Minimum number of Libre <= 1
    # model.add_at_most_one(tasks[(e, d, activities[5])] for e in employees)

# 3️⃣ Limitation of consecutive days on "Tél" or "Rens" activity.
# Takes account the different weeks
for e in employees:
    for w in weeks:
        for d in range(5 * w, 5 * (w + 1) - 2):
            # Maximum 2 consecutive days of "Tél"
            model.add(sum(tasks[(e, d + i, activities[0])] for i in range(3)) <= 2)
        for d in range(5 * w, 5 * (w + 1) - 1):
            # Maximum 1 consecutive days of "Rens"
            model.add(sum(tasks[(e, d + i, activities[1])] for i in range(2)) <= 1)
        # Maximum 3 days of "Tél" in the week
        model.add(sum(tasks[(e, 5 * w + i, activities[0])] for i in range(5)) <= 3)
        # Maximum 2 days of "Rens" in the week
        model.add(sum(tasks[(e, 5 * w + i, activities[1])] for i in range(5)) <= 2 * 1)
        # Maximum 1 day of "Récla" in the week
        model.add(sum(tasks[(e, 5 * w + i, activities[2])] for i in range(5)) <=  1)
        # Maximum 1 day of "Dérog" in the week
        model.add(sum(tasks[(e, 5 * w + i, activities[3])] for i in range(5)) <=  1)
        # Maximum 1 day of "Imp" in the week
        model.add(sum(tasks[(e, 5 * w + i, activities[4])] for i in range(5)) <=  1)

# 4️⃣ At least 3 different activities per week
for e in employees:
    for w in weeks:
        different_activities = []
        for a in activities[:-1]:  # Exclude "Libre" activity
            worked_this_activity = model.NewBoolVar(f"worked_{e}_week{w}_{a}")
            model.Add(sum(tasks[(e, 5 * w + i, a)] for i in range(5)) >= 1).OnlyEnforceIf(worked_this_activity)
            model.Add(sum(tasks[(e, 5 * w + i, a)] for i in range(5)) == 0).OnlyEnforceIf(worked_this_activity.Not())
            different_activities.append(worked_this_activity)
        model.Add(sum(different_activities) >= 3)

# 5️⃣ At least once "Tél" and once "Rens" per week
for e in employees:
    for w in weeks:
        worked_tel = model.NewBoolVar(f"work_tel_{e}_{w}")
        model.Add(sum(tasks[(e, 5 * w + i, activities[0])] for i in range(5)) >= 1).OnlyEnforceIf(worked_tel)
        model.Add(sum(tasks[(e, 5 * w + i, activities[0])] for i in range(5)) == 0).OnlyEnforceIf(worked_tel.Not())
        worked_rens = model.NewBoolVar(f"work_rens_{e}_{w}")
        model.Add(sum(tasks[(e, 5 * w + i, activities[1])] for i in range(5)) >= 1).OnlyEnforceIf(worked_rens)
        model.Add(sum(tasks[(e, 5 * w + i, activities[1])] for i in range(5)) == 0).OnlyEnforceIf(worked_rens.Not())
        model.add_bool_and([worked_tel, worked_rens])

# 6️⃣ Exactly one "Libre" every 2 weeks
# for e in employees:
#     for w in range(len(weeks) - 1):
#         worked_libre_in_2_weeks = model.NewBoolVar(f"work_libre_{e}_{w}")
#         model.Add(sum(tasks[(e, 5 * w + i, activities[5])] for i in range(10)) >= 1).OnlyEnforceIf(worked_libre_in_2_weeks)
#         model.Add(sum(tasks[(e, 5 * w + i, activities[5])] for i in range(10)) == 0).OnlyEnforceIf(worked_libre_in_2_weeks.Not())
#         model.add(worked_libre_in_2_weeks == 1)
#         worked_libre_in_2_weeks = model.NewBoolVar(f"work_libre_{e}_{w}")
#         model.Add(sum(tasks[(e, 5 * w + i, activities[5])] for i in range(10)) >= 1).OnlyEnforceIf(worked_libre_in_2_weeks)
#         model.Add(sum(tasks[(e, 5 * w + i, activities[5])] for i in range(10)) == 0).OnlyEnforceIf(worked_libre_in_2_weeks.Not())
#         model.add(worked_libre_in_2_weeks == 1)

#  Same activity on morning and afternoon shifts (To Study)
# for e in employees:
#     for d in days:
#         # Variable binaire : 1 si les activités matin et après-midi sont différentes
#         different_activity = model.NewBoolVar(f"different_{e}_{d}")

#         # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
#         same_activity_bools = []
#         for a in activities:
#             both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
#             model.AddBoolAnd([tasks[(e, d, shifts[0], a)], tasks[(e, d, shifts[1], a)]]).OnlyEnforceIf(both_same)
#             model.AddBoolOr([tasks[(e, d, shifts[0], a)].Not(), tasks[(e, d, shifts[1], a)].Not()]).OnlyEnforceIf(both_same.Not())
#             same_activity_bools.append(both_same)

#         # Si au moins une activité est identique sur la journée, alors different_activity = 0
#         model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
#         # Sinon, different_activity = 1
#         model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

#         # On peut ajouter une pénalité si on veut minimiser les changements d'activité
#         # model.Minimize(different_activity)

# 7️⃣ Constraints on activities (To Study)
# for e in employees:
#     for d in days:
#         for s in shifts:
#             # If employee e works on "Dérog" on day d and shift s, then they cannot work on "Dérog" the next day
#             if d < len(days) - 1:
#                 model.add_implication(tasks[(e, d, s, activities[2])], tasks[(e, d + 1, s, activities[2])].Not())
#             # If employee e works on "Récla" on day d and shift s, then they cannot work on "Récla" the next day
#             if d < len(days) - 1:
#                 model.add_implication(tasks[(e, d, s, activities[3])], tasks[(e, d + 1, s, activities[3])].Not())


# 8️⃣ 9️⃣ 🔟



# -----------------------------------
# Creates the solver and solve.
# -----------------------------------

solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first 'n' solutions.
solution_limit = 1
solution_printer = PartialSolutionPrinter(employees, days, activities, tasks, solution_limit)
status = solver.solve(model, solution_printer)

if status == cp_model.INFEASIBLE:
    print("Pas de solution !")

else:
    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")
    print(f"  - solutions found: {solution_printer.solutionCount()}")

Solution 1
       A     B     C     D     E     F     G     H     I     J     K     L     M     N
 1 →   *   Tél   Tél   Tél   Tél     *   Rens  Tél   Rens  Rens  Récla Imp   Dérog Dérog 
 2 →   *   Rens  Rens  Dérog Rens  Dérog Tél   Tél   Tél   Imp   Tél   Récla Tél   Tél   
 3 → Rens  Tél   Tél   Tél   Tél   Tél   Dérog   *   Imp   Récla Rens  Rens  Rens  Rens  
 4 → Dérog Rens    *   Rens  Rens  Tél   Tél   Imp   Tél   Tél   Tél   Tél   Récla Tél   
 5 → Tél   Dérog Dérog Tél   Dérog Rens  Tél   Rens  Récla Tél   Imp   Tél   Rens  Tél   

 6 → Libre Tél   Tél   Tél   Tél   Tél   Dérog Dérog Rens  Rens  Rens  Récla Imp   Dérog 
 7 → Rens  Dérog Dérog Rens  Rens  Tél   Tél   Tél   Tél   Imp   Tél   Tél   Récla Tél   
 8 → Tél   Tél   Tél   Dérog Tél   Rens  Rens  Rens  Imp   Tél   Récla Dérog Rens  Rens  
 9 → Rens  Rens  Libre Tél   Dérog Dérog Tél   Tél   Tél   Récla Imp   Rens  Tél   Tél   
10 → Dérog   *   Rens  Rens  Rens  Tél   Tél   Tél   Récla Tél   Tél   Imp   Tél   Tél   



##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Second Test : CA Planning with two shifts a day (Real Case)
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    test
    </h3>
</div>

In [5]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, employees, days, shifts, activities, tasks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._employees = employees
        self._days = days
        self._shifts = shifts
        self._activities = activities
        self._tasks = tasks
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        print("     " + "           ".join(self._employees))

        for d in self._days:
            st = f"{d + 1:2} → "
            st = ""
            for e in self._employees:
                for s in self._shifts:
                    is_working = False
                    for a in self._activities:
                        if self.value(self._tasks[(e, d, s, a)]):
                            st += f"{a:5} "
                            is_working = True
                    if not is_working:
                        st += "  *   "
            print(st)
            if (d + 1) % 5 == 0:
                print()
        # print()
        # self.print_employees_statistics()
        # print()
        # self.print_days_statistics()
        # print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def print_days_statistics(self):
        for d in self._days:
            for i, s in enumerate(self._shifts):
                st = f"{d + 1:2} "
                if i:
                    st += "Matin      → "
                else:
                    st += "Après-midi → "
                total = 0
                for a in self._activities:
                    nb = sum(self.value(self._tasks[(e, d, s, a)]) for e in self._employees)
                    total += nb
                    st += f"{a}:{nb} "
                print(st + f" (nb activities: {total})")

    def print_employees_statistics(self):
        for e in self._employees:
            st = f"{e} → "
            total = 0
            for a in self._activities:
                nb = sum(self.value(self._tasks[(e, d, s, a)]) for d in self._days for s in self._shifts)
                total += nb
                st += f"{a}:{nb} "
            print(st + f" (nb activities: {total})")

    def solutionCount(self):
        return self._solution_count


# Data.
n_employees = 14
n_weeks = 2
n_days = 5 * n_weeks

employees = [chr(i) for i in range(65, 65 + n_employees)]
weeks = range(n_weeks)
days = range(n_days)
shifts = ["Matin", "Après-midi"]
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp", "Libre"]
ACT_TEL, ACT_RENS, ACT_DEROG, ACT_RECLA, ACT_IMP, ACT_LIBRE = range(len(activities))

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# tasks[(e, d, s, a)]: employee 'e' works day 'd' on shift 's' on activity 'a'.
tasks = {}
for e in employees:
    for d in days:
        for s in shifts:
            for a in activities:
                tasks[(e, d, s, a)] = model.new_bool_var(f"task_{e}_{d}_{s}_{a}")



# ------------------- #
#                     #
#     CONSTRAINTS     #
#                     #
# ------------------- #

# Days Off
is_present = {item : [True] * 2 * len(days) for item in employees}
is_present['A'][0] = False
is_present['A'][1] = False
is_present['B'][9] = False
is_present['C'][2] = False
is_present['C'][3] = False
is_present['F'][0] = False
is_present['H'][2] = False
# is_present['O'] = [False] * len(days) * len(shifts)

# 1️⃣ Each employee must have exactly one activity per shift.
for e in employees:
    for d in days:
        for s in shifts:
            if is_present[e][d]:
                # Employee is present this shift
                model.add(sum(tasks[(e, d, s, a)] for a in activities) == 1)
            else:
                # Employee is not present this shift
                model.add(sum(tasks[(e, d, s, a)] for a in activities) == 0)

# 2️⃣ Minimum / Maximum number of employees per activity.
# To be defined according to how many people are available.
for d in days:
    for s in shifts:
        # Minimum number of Tél >= 5
        model.add(sum(tasks[(e, d, s, activities[ACT_TEL])] for e in employees) >= 5)
        # Minimum number of Rens >= 3
        model.add(sum(tasks[(e, d, s, activities[ACT_RENS])] for e in employees) >= 3)
        # Minimum number of Dérog >= 1
        model.add_at_least_one(tasks[(e, d, s, activities[ACT_DEROG])] for e in employees)
        # Minimum number of Récla == 1
        model.add_exactly_one(tasks[(e, d, s, activities[ACT_RECLA])] for e in employees)
        # Minimum number of Imp == 1
        model.add_exactly_one(tasks[(e, d, s, activities[ACT_IMP])] for e in employees)
        # Minimum number of Libre <= 1
        model.add_at_most_one(tasks[(e, d, s, activities[ACT_LIBRE])] for e in employees)

# 3️⃣ Limitation of consecutive days on "Tél" or "Rens" activity.
# Takes account the different weeks
for e in employees:
    for w in weeks:
        for d in range(5 * w, 5 * (w + 1) - 2):
            # Maximum 2 consecutive days of "Tél"
            model.add(sum(tasks[(e, d + i, s, activities[ACT_TEL])] for i in range(3) for s in shifts) <= 2 * len(shifts))
        for d in range(5 * w, 5 * (w + 1) - 1):
            # Maximum 1 consecutive days of "Rens"
            model.add(sum(tasks[(e, d + i, s, activities[ACT_RENS])] for i in range(2) for s in shifts) <= len(shifts))
        # Maximum 3 days of "Tél" in the week
        model.add(sum(tasks[(e, 5 * w + i, s, activities[ACT_TEL])] for i in range(5) for s in shifts) <= 3 * len(shifts))
        # Maximum 2 days of "Rens" in the week
        model.add(sum(tasks[(e, 5 * w + i, s, activities[ACT_RENS])] for i in range(5) for s in shifts) <= 2 * len(shifts))
        # Maximum 1 day of "Dérog" in the week
        model.add(sum(tasks[(e, 5 * w + i, s, activities[ACT_DEROG])] for i in range(5) for s in shifts) <=  len(shifts))
        # Maximum 1 day of "Récla" in the week
        model.add(sum(tasks[(e, 5 * w + i, s, activities[ACT_RECLA])] for i in range(5) for s in shifts) <=  len(shifts))
        # Maximum 1 day of "Imp" in the week
        model.add(sum(tasks[(e, 5 * w + i, s, activities[ACT_IMP])] for i in range(5) for s in shifts) <=  len(shifts))

# 4️⃣ At least 3 different activities per week
for e in employees:
    for w in weeks:
        different_activities = []
        for a in activities[:-1]:  # Exclude "Libre" activity
            worked_this_activity = model.NewBoolVar(f"worked_{e}_week{w}_{a}")
            model.Add(sum(tasks[(e, 5 * w + i, s, a)] for i in range(5) for s in shifts) >= 1).OnlyEnforceIf(worked_this_activity)
            model.Add(sum(tasks[(e, 5 * w + i, s, a)] for i in range(5) for s in shifts) == 0).OnlyEnforceIf(worked_this_activity.Not())
            different_activities.append(worked_this_activity)
        model.Add(sum(different_activities) >= 3)

# 5️⃣ At least once "Tél" and once "Rens" per week
for e in employees:
    for w in weeks:
        worked_tel = model.NewBoolVar(f"work_tel_{e}_{w}")
        model.Add(sum(tasks[(e, 5 * w + i, s, activities[ACT_TEL])] for i in range(5) for s in shifts) >= 1).OnlyEnforceIf(worked_tel)
        model.Add(sum(tasks[(e, 5 * w + i, s, activities[ACT_TEL])] for i in range(5) for s in shifts) == 0).OnlyEnforceIf(worked_tel.Not())
        worked_rens = model.NewBoolVar(f"work_rens_{e}_{w}")
        model.Add(sum(tasks[(e, 5 * w + i, s, activities[ACT_RENS])] for i in range(5) for s in shifts) >= 1).OnlyEnforceIf(worked_rens)
        model.Add(sum(tasks[(e, 5 * w + i, s, activities[ACT_RENS])] for i in range(5) for s in shifts) == 0).OnlyEnforceIf(worked_rens.Not())
        model.add_bool_and([worked_tel, worked_rens])

# 6️⃣ Exactly one "Libre" every 2 weeks
# for e in employees:
#     for w in range(len(weeks) - 1):
#         worked_libre_in_2_weeks = model.NewBoolVar(f"work_libre_{e}_{w}")
#         model.Add(sum(tasks[(e, 5 * w + i, s, activities[5])] for i in range(10) for s in shifts) >= 1).OnlyEnforceIf(worked_libre_in_2_weeks)
#         model.Add(sum(tasks[(e, 5 * w + i, s, activities[5])] for i in range(10) for s in shifts) == 0).OnlyEnforceIf(worked_libre_in_2_weeks.Not())
#         model.add(worked_libre_in_2_weeks == 1)
#         worked_libre_in_2_weeks = model.NewBoolVar(f"work_libre_{e}_{w}")
#         model.Add(sum(tasks[(e, 5 * w + i, s, activities[5])] for i in range(10) for s in shifts) >= 1).OnlyEnforceIf(worked_libre_in_2_weeks)
#         model.Add(sum(tasks[(e, 5 * w + i, s, activities[5])] for i in range(10) for s in shifts) == 0).OnlyEnforceIf(worked_libre_in_2_weeks.Not())
#         model.add(worked_libre_in_2_weeks == 1)

# 7️⃣ Same activity on morning and afternoon shifts (To Study)
# for e in employees:
#     for d in days:
#         # Variable binaire : 1 si les activités matin et après-midi sont différentes
#         different_activity = model.NewBoolVar(f"different_{e}_{d}")

#         # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
#         same_activity_bools = []
#         for a in activities:
#             both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
#             model.AddBoolAnd([tasks[(e, d, shifts[0], a)], tasks[(e, d, shifts[1], a)]]).OnlyEnforceIf(both_same)
#             model.AddBoolOr([tasks[(e, d, shifts[0], a)].Not(), tasks[(e, d, shifts[1], a)].Not()]).OnlyEnforceIf(both_same.Not())
#             same_activity_bools.append(both_same)

#         # Si au moins une activité est identique sur la journée, alors different_activity = 0
#         model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
#         # Sinon, different_activity = 1
#         model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

#         # On peut ajouter une pénalité si on veut minimiser les changements d'activité
#         model.Minimize(different_activity)

penalties = []
for e in employees:
    for d in days:
        # Variable binaire : 1 si les activités matin et après-midi sont différentes
        different_activity = model.NewBoolVar(f"different_{e}_{d}")

        # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
        same_activity_bools = []
        for a in activities:
            both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
            model.AddBoolAnd([tasks[e, d, shifts[0], a], tasks[e, d, shifts[1], a]]).OnlyEnforceIf(both_same)
            model.AddBoolOr([tasks[e, d, shifts[0], a].Not(), tasks[e, d, shifts[1], a].Not()]).OnlyEnforceIf(both_same.Not())
            same_activity_bools.append(both_same)

        # Si au moins une activité est identique sur la journée, alors different_activity = 0
        model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
        # Sinon, different_activity = 1
        model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)
        penalties.append(different_activity)

# On minimise le nombre total de changements d'activité sur la semaine
model.Minimize(sum(penalties))

# 8️⃣ 9️⃣ 🔟



# -----------------------------------
# Creates the solver and solve.
# -----------------------------------

solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# solver.parameters.max_time_in_seconds = 10
# solver.parameters.num_search_workers = 8
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first 'n' solutions.
solution_limit = 1
solution_printer = PartialSolutionPrinter(employees, days, shifts, activities, tasks, solution_limit)
status = solver.solve(model, solution_printer)

if status == cp_model.INFEASIBLE:
    print("Pas de solution !")

else:
    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")
    print(f"  - solutions found: {solution_printer.solutionCount()}")

Solution 1
     A           B           C           D           E           F           G           H           I           J           K           L           M           N
  *     *   Rens  Tél   Tél   Dérog Rens  Tél   Tél   Tél     *     *   Tél   Libre Récla Tél   Rens  Tél   Tél   Rens  Tél   Rens  Libre Imp   Imp   Récla Dérog Rens  
  *     *   Dérog Tél   Rens  Tél   Rens  Tél   Tél   Rens  Tél   Dérog Tél   Libre Rens  Tél   Tél   Rens  Tél   Rens  Imp   Tél   Récla Imp   Dérog Rens  Libre Récla 
Tél   Tél   Tél   Rens    *     *   Rens  Imp   Rens  Récla Récla Tél   Tél   Dérog   *     *   Dérog Rens  Imp   Rens  Rens  Dérog Tél   Tél   Tél   Tél   Libre Tél   
Tél   Rens  Rens  Tél     *     *   Tél   Rens  Tél   Dérog Rens  Imp   Imp   Tél   Rens  Dérog Rens  Tél   Tél   Récla Récla Tél   Dérog Rens  Libre Libre Tél   Tél   
Tél   Imp   Rens  Dérog Tél   Rens  Dérog Tél   Tél   Rens  Rens  Tél   Récla Rens  Rens  Dérog Imp   Récla Rens  Dérog Rens  Tél   Tél   Tél   Tél   